In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
df = pd.read_csv("shuffled-full-set-hashed.csv", header=None)

In [2]:
encoder = pickle.load(open('../server/webservice/pickles/encoder.pkl', 'rb'))
smaller_vectorizer = pickle.load(open('./smaller_vectorizer.pkl', 'rb'))
new_lsa = pickle.load(open('./new_lsa.pkl', 'rb'))

In [3]:
output_size = len(encoder.classes_)
sample = df[0:10000]
sample.dropna(inplace=True)
sample_y = sample[0]

def get_label_vect(label):
    vect = [0] * output_size
    vect[label - 1] = 1
    return vect
encoded_y = np.array([get_label_vect(l) for l in encoder.transform(sample_y)])
encoded_y

/Users/boaz.reisman/.virtualenvs/datascience/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [4]:
chunk_size = 20
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        new_chunk = l[i:i+n]
        if len(new_chunk) < n:
            new_chunk = ([''] * (n - len(new_chunk))) + new_chunk
        yield new_chunk
        
doc_arr = [x for x in sample[1]]
doc_arr[0].split(' ')
doc_words = [doc.split(' ') for doc in doc_arr]


In [5]:
training_set = []
len(doc_words), len(encoded_y)
for i in range(len(doc_words)):
    for chunk in chunks(doc_words[i], chunk_size):
        training_set.append((encoded_y[i], chunk))
        
training_vects = []        

In [6]:
len(training_set)

170380

In [7]:
import random
random.shuffle(training_set)

In [8]:
# Now for each word, get the tfidf vector
def word_2_vect(word):
    return new_lsa.transform(smaller_vectorizer.transform(pd.Series([word])))[0]

i = 0
for label, chunk in training_set:
    training_vects.append((label, [word_2_vect(word) for word in chunk]))
    if i%500 == 0:
        print('{} of {}'.format(i, len(training_set)))
    i += 1

training_vects[0]

0 of 170380
500 of 170380
1000 of 170380
1500 of 170380
2000 of 170380
2500 of 170380
3000 of 170380
3500 of 170380
4000 of 170380
4500 of 170380
5000 of 170380
5500 of 170380
6000 of 170380
6500 of 170380
7000 of 170380
7500 of 170380
8000 of 170380
8500 of 170380
9000 of 170380
9500 of 170380
10000 of 170380
10500 of 170380
11000 of 170380
11500 of 170380
12000 of 170380
12500 of 170380
13000 of 170380
13500 of 170380
14000 of 170380
14500 of 170380
15000 of 170380
15500 of 170380
16000 of 170380
16500 of 170380
17000 of 170380
17500 of 170380
18000 of 170380
18500 of 170380
19000 of 170380
19500 of 170380
20000 of 170380
20500 of 170380
21000 of 170380
21500 of 170380
22000 of 170380
22500 of 170380
23000 of 170380
23500 of 170380
24000 of 170380
24500 of 170380
25000 of 170380
25500 of 170380
26000 of 170380
26500 of 170380
27000 of 170380
27500 of 170380
28000 of 170380
28500 of 170380
29000 of 170380
29500 of 170380
30000 of 170380
30500 of 170380
31000 of 170380
31500 of 170380


(array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 [array([ 0.1434493 ,  0.02206802, -0.07441233,  0.02167092,  0.00049866,
         -0.09899638,  0.01377018, -0.00330719,  0.03428012,  0.11761893,
         -0.1722495 ,  0.01206386, -0.0321017 , -0.07461794,  0.10068704,
          0.12426958, -0.02082334, -0.03588277, -0.00687876,  0.0053177 ,
          0.07970418,  0.00292972, -0.00854009, -0.08107421, -0.02387904,
          0.07992019, -0.08593847, -0.00419079, -0.00518038,  0.00845898,
          0.02266443, -0.04533478,  0.0849636 ,  0.06195972,  0.16432809,
          0.03302996, -0.09877219, -0.03342144, -0.03901842,  0.04955141,
          0.11901986, -0.11069657,  0.0300322 ,  0.03475586,  0.0606498 ,
         -0.14606519, -0.06019993,  0.09585275, -0.02104146, -0.13177359,
         -0.15374803, -0.06068371,  0.03552494,  0.01796003,  0.06610803,
         -0.00524239,  0.10040445,  0.19491042,  0.13472122, -0.02049665,
          0.15770875,  0.07092995, -0.15797417, -0.25890489,

In [10]:

len(training_vects)

170380

In [13]:
i = 0
for ch in chunks(training_vects, 80000):
    pickle.dump(ch, open('training_vects_big_{}.pkl'.format(i), 'wb'))
    i += 1

In [14]:
lstm_size = 256
lstm_layers = 2
batch_size = 500
learning_rate = 0.001

In [15]:
# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder("float", [None, chunk_size, 100])
    labels_ = tf.placeholder(tf.int32, [None, output_size])
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [16]:
with graph.as_default():
    with tf.name_scope("RNN_layers"):
        def lstm_cell():
            # Your basic LSTM cell
            lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size, reuse=tf.get_variable_scope().reuse)
            # Add dropout to the cell
            return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

        # Stack up multiple LSTM layers, for deep learning
        cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])

        # Getting an initial state of all zeros
        initial_state = cell.zero_state(batch_size, tf.float32)

W1003 20:04:28.423163 140736238351232 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1003 20:04:28.424947 140736238351232 deprecation.py:323] From <ipython-input-16-ab0169ed8749>:5: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W1003 20:04:28.431388 140736238351232 deprecation.py:323] From <ipython-input-16-ab0169ed8749>:10: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instruction

In [17]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs_,
                                             initial_state=initial_state)

W1003 20:04:34.272701 140736238351232 deprecation.py:323] From <ipython-input-17-94a791623b59>:3: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W1003 20:04:34.592230 140736238351232 deprecation.py:506] From /Users/boaz.reisman/.virtualenvs/datascience/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1003 20:04:34.602071 140736238351232 deprecation.py:506] From /Users/boaz.reisman/.virtualenvs/datascience/lib/python3.6/site-packages/tensorflow/python/ops/rnn_cell_impl.py:738: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will 

In [18]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], output_size, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

W1003 20:04:43.100605 140736238351232 deprecation.py:323] From /Users/boaz.reisman/.virtualenvs/datascience/lib/python3.6/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [19]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [20]:
y = np.array([tup[0] for tup in training_vects])
y.shape

(170380, 14)

In [21]:
x = np.array([tup[1] for tup in training_vects])
x.shape

(170380, 20, 100)

In [22]:
def get_batches(x, y, batch_size=300):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [23]:
train_prop = 0.8

set_size = x.shape[0]

split_idx = int(set_size*0.8)
train_x, rest_x = x[:split_idx], x[split_idx:]
train_y, rest_y = y[:split_idx], y[split_idx:]

val_prop = 0.5
rest_size = rest_x.shape[0]
val_idx = int(val_prop * rest_size)
val_x, test_x = rest_x[:val_idx], rest_x[val_idx:]
val_y, test_y = rest_y[:val_idx], rest_y[val_idx:]


In [24]:
train_y.shape, y.shape, val_y.shape, rest_y.shape

((136304, 14), (170380, 14), (17038, 14), (34076, 14))

In [25]:
val_y[:10]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [27]:
epochs = 10

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (batch_x, batch_y) in enumerate(get_batches(train_x, train_y, batch_size), 1):

            feed = {inputs_: batch_x,
                    labels_: batch_y,
                    keep_prob: 0.5,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)

            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                
                for batch_val_x, batch_val_y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: batch_val_x,
                            labels_: batch_val_y,
                            keep_prob: 1, 
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)

                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))

            
            iteration +=1
            saver.save(sess, "checkpoints/final_sentiment.ckpt")
    saver.save(sess, "checkpoints/final_sentiment.ckpt")

Epoch: 0/10 Iteration: 5 Train loss: 0.195
Epoch: 0/10 Iteration: 10 Train loss: 0.061
Epoch: 0/10 Iteration: 15 Train loss: 0.062
Epoch: 0/10 Iteration: 20 Train loss: 0.061
Epoch: 0/10 Iteration: 25 Train loss: 0.058
Val acc: 0.929
Epoch: 0/10 Iteration: 30 Train loss: 0.060
Epoch: 0/10 Iteration: 35 Train loss: 0.060
Epoch: 0/10 Iteration: 40 Train loss: 0.058
Epoch: 0/10 Iteration: 45 Train loss: 0.057
Epoch: 0/10 Iteration: 50 Train loss: 0.059
Val acc: 0.929
Epoch: 0/10 Iteration: 55 Train loss: 0.057
Epoch: 0/10 Iteration: 60 Train loss: 0.057
Epoch: 0/10 Iteration: 65 Train loss: 0.057
Epoch: 0/10 Iteration: 70 Train loss: 0.058
Epoch: 0/10 Iteration: 75 Train loss: 0.058
Val acc: 0.929
Epoch: 0/10 Iteration: 80 Train loss: 0.058
Epoch: 0/10 Iteration: 85 Train loss: 0.057
Epoch: 0/10 Iteration: 90 Train loss: 0.057
Epoch: 0/10 Iteration: 95 Train loss: 0.056
Epoch: 0/10 Iteration: 100 Train loss: 0.057
Val acc: 0.929
Epoch: 0/10 Iteration: 105 Train loss: 0.057
Epoch: 0/10 Ite

Epoch: 3/10 Iteration: 865 Train loss: 0.045
Epoch: 3/10 Iteration: 870 Train loss: 0.041
Epoch: 3/10 Iteration: 875 Train loss: 0.041
Val acc: 0.947
Epoch: 3/10 Iteration: 880 Train loss: 0.043
Epoch: 3/10 Iteration: 885 Train loss: 0.043
Epoch: 3/10 Iteration: 890 Train loss: 0.040
Epoch: 3/10 Iteration: 895 Train loss: 0.040
Epoch: 3/10 Iteration: 900 Train loss: 0.042
Val acc: 0.948
Epoch: 3/10 Iteration: 905 Train loss: 0.041
Epoch: 3/10 Iteration: 910 Train loss: 0.043
Epoch: 3/10 Iteration: 915 Train loss: 0.040
Epoch: 3/10 Iteration: 920 Train loss: 0.039
Epoch: 3/10 Iteration: 925 Train loss: 0.041
Val acc: 0.948
Epoch: 3/10 Iteration: 930 Train loss: 0.041
Epoch: 3/10 Iteration: 935 Train loss: 0.041
Epoch: 3/10 Iteration: 940 Train loss: 0.039
Epoch: 3/10 Iteration: 945 Train loss: 0.042
Epoch: 3/10 Iteration: 950 Train loss: 0.043
Val acc: 0.949
Epoch: 3/10 Iteration: 955 Train loss: 0.040
Epoch: 3/10 Iteration: 960 Train loss: 0.040
Epoch: 3/10 Iteration: 965 Train loss: 0

Val acc: 0.952
Epoch: 6/10 Iteration: 1705 Train loss: 0.037
Epoch: 6/10 Iteration: 1710 Train loss: 0.036
Epoch: 6/10 Iteration: 1715 Train loss: 0.037
Epoch: 6/10 Iteration: 1720 Train loss: 0.037
Epoch: 6/10 Iteration: 1725 Train loss: 0.037
Val acc: 0.952
Epoch: 6/10 Iteration: 1730 Train loss: 0.037
Epoch: 6/10 Iteration: 1735 Train loss: 0.036
Epoch: 6/10 Iteration: 1740 Train loss: 0.040
Epoch: 6/10 Iteration: 1745 Train loss: 0.037
Epoch: 6/10 Iteration: 1750 Train loss: 0.036
Val acc: 0.952
Epoch: 6/10 Iteration: 1755 Train loss: 0.034
Epoch: 6/10 Iteration: 1760 Train loss: 0.038
Epoch: 6/10 Iteration: 1765 Train loss: 0.038
Epoch: 6/10 Iteration: 1770 Train loss: 0.041
Epoch: 6/10 Iteration: 1775 Train loss: 0.036
Val acc: 0.952
Epoch: 6/10 Iteration: 1780 Train loss: 0.040
Epoch: 6/10 Iteration: 1785 Train loss: 0.040
Epoch: 6/10 Iteration: 1790 Train loss: 0.036
Epoch: 6/10 Iteration: 1795 Train loss: 0.039
Epoch: 6/10 Iteration: 1800 Train loss: 0.038
Val acc: 0.952
Epoch

Epoch: 9/10 Iteration: 2545 Train loss: 0.037
Epoch: 9/10 Iteration: 2550 Train loss: 0.036
Val acc: 0.953
Epoch: 9/10 Iteration: 2555 Train loss: 0.035
Epoch: 9/10 Iteration: 2560 Train loss: 0.034
Epoch: 9/10 Iteration: 2565 Train loss: 0.041
Epoch: 9/10 Iteration: 2570 Train loss: 0.036
Epoch: 9/10 Iteration: 2575 Train loss: 0.038
Val acc: 0.953
Epoch: 9/10 Iteration: 2580 Train loss: 0.038
Epoch: 9/10 Iteration: 2585 Train loss: 0.038
Epoch: 9/10 Iteration: 2590 Train loss: 0.040
Epoch: 9/10 Iteration: 2595 Train loss: 0.036
Epoch: 9/10 Iteration: 2600 Train loss: 0.037
Val acc: 0.953
Epoch: 9/10 Iteration: 2605 Train loss: 0.034
Epoch: 9/10 Iteration: 2610 Train loss: 0.036
Epoch: 9/10 Iteration: 2615 Train loss: 0.038
Epoch: 9/10 Iteration: 2620 Train loss: 0.033
Epoch: 9/10 Iteration: 2625 Train loss: 0.035
Val acc: 0.954
Epoch: 9/10 Iteration: 2630 Train loss: 0.038
Epoch: 9/10 Iteration: 2635 Train loss: 0.035
Epoch: 9/10 Iteration: 2640 Train loss: 0.038
Epoch: 9/10 Iteratio

In [179]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, "checkpoints/final_sentiment.ckpt")
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (bat_test_x, bat_test_y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: bat_test_x,
                labels_: bat_test_y,
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

Test accuracy: 0.954


In [192]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, "checkpoints/final_sentiment.ckpt")
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (bat_test_x, bat_test_y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: bat_test_x,
                labels_: bat_test_y,
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state, pdctns, cor_p = sess.run([accuracy, final_state, predictions, correct_pred], feed_dict=feed)
        print(ii)
        print(cor_p[0])
        print(max_prediction(pdctns))
        print(bat_test_y[0])
        print(all(max_prediction(pdctns) == bat_test_y[0]))
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

1
[ True  True False  True  True  True  True  True  True  True  True  True
  True  True]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0]
True
2
[ True False  True  True  True  True  True  True  True  True  True  True
  True  True]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0]
False
3
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0]
True
4
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0]
True
5
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0]
True
6
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0]
True
7
[ True  True  True  True  True  True  True  True  True  True  True  True


In [189]:
words = 'cd50e861f48b 6ca2dd348663 d38820625542 f07761418345 5948001254b3 9997460cf372 3f7a70f3474f 6bc122aa4b06 b4f1b88bd3f3 25c57acdf805 54709b24b45f 51fc8b56e14f ff714b9fa7fd 798fe9915030 2d5d7bc2df91 6b583fe13e39 8e93a2273a93 3d877a3bc4f0 ff1c26ea0b6f d9ef68daef4c 9a42ead47d1c 5cde1ed4f9b0 a20e78c41cf9 dec88250479b ecee69844b4e 87b8193a0183 0c470ee92bb5 cd50f04925dd 878460b4304e b4f1b88bd3f3 133d46f7ed38 d38820625542 f682089485c3 96776b42c1d5 51fc8b56e14f 3f7a70f3474f d911e9441d32 c16651ffa1b7 9374c105ef84 f8b0c07e306c d38820625542 e0a08df8ec4c f8dab246f73f 878460b4304e 3f7a70f3474f b4f1b88bd3f3 6ce6cc5a3203 b4f1b88bd3f3 ddf4525e90e3 55b85f1ee56e c63534ebfa82 878460b4304e 9448e1c70dec 1932d6ce3497 8243aca146b2 2ea49cf89745 ff714b9fa7fd e4dad7cb07b6 ff714b9fa7fd 549330b9e320 f7bb594ff117 e63fa74d3c8b df03d288b8b6 cd50e861f48b f7bb594ff117 a1bb6b4223d9 ba02159e05b1 37ac79620fc6 1087de55f102 80948b43ec70 d9fc2cff7e21 abca9d18fae2 6cfef272bdb9 9d83e581af4b cc9e05bc2a86 cbd0a3abbc34 69954839bb65 cd50f04925dd 431392fb12c7 ecee69844b4e 36e7aa72ffe1 0b3f3d23bd37 8cb71bb0ee27 ff714b9fa7fd a87473e878c1 a1bb6b4223d9 f7bb594ff117 dff2d27792d5 ff714b9fa7fd 83f8b75d8fe3 22f8fa324aed 58b1c79289d7 5948001254b3 9997460cf372 da9ad7407226 a263b63bc282 011113964d37 da9ad7407226 b73e657498f2 17c11ac4749f ee8125ecd585 0cbca93be301 ea95010f229b b13cff832743 cf4fc632eed2 b4f1b88bd3f3 6b343f522f78 4ea48dbbe14e 5f9c2ac954be 61e151399c9d 7b477e3d63d2 cdc95c39cac8 f07761418345 22fa1184be26 d9ef68daef4c d944e076fa92 f7ae6f8257da 87b8193a0183 11269ab19093 6cfef272bdb9 8ebb4fffd292 d38820625542 6469b7eeb029 98b0900e06f4 1b6c95839a6d 44c1e122f3f0 d38820625542 cfd22ba194a9 828e5c947b2b 7860028b1d17 6dbd5ceef3d1 f60bd1bc4fe5 05b8cb495fc2 9d83e581af4b d38820625542 7d9e333a86da 6cfef272bdb9 9d83e581af4b 133d46f7ed38 1087de55f102 ef4ba44cdf5f b02eb907dd1a 35991d8609e2 29f6e5802d4f 133d46f7ed38 874b796362b6 e943e5e5b779 4d28878de1db d1c3631d621b 7d9e333a86da d38820625542 133d46f7ed38 1b7657be8986 1ab34730c1e0 f7bb594ff117 ff714b9fa7fd 8502c0087099 58b1c79289d7 cd50f04925dd ecee69844b4e 431392fb12c7 1b3e5dc39eb9 586242498a88 5680bf4cc121 69c87281a156 c8207fafe699 cd50f04925dd ff714b9fa7fd 8502c0087099 ecee69844b4e 8cb71bb0ee27 386e8b20726f a1bb6b4223d9 6bf9c0cb01b4 142d16db61dd 60d18132df81 dab42b5e7528 259218088611 135307dba198 b73e657498f2 26f7353edc2e cd50f04925dd d38820625542 133d46f7ed38 f7bb594ff117 ecee69844b4e 1ab34730c1e0 e943e5e5b779 a87473e878c1 57d5616683d8 54fb27d07530 afab4a5c0f58 83da9eb0a417 619c02c2888a 11269ab19093 fb82195dee42 b7a0f56f6ce8 2d5d7bc2df91 6b583fe13e39 798fe9915030 586242498a88 0662d94b3d3b b7ab56536ec4 f54a40468e55 14bc74b659ac 7498a1e9d9d7 ac624879ac84 6bc122aa4b06 868759570f1a 8d21095e8690 4cc4edfe0691 a28d2053ff0a abca9d18fae2 586242498a88 1446e5509927 7ec02e30a5b3 00c51a5afedc b9699ce57810 ba02159e05b1 f0666bdbc8a5 e7e059c82399 6bc963b8a2b5 6d3eb69fcdd2 ae28415398b8 25c57acdf805 bce6be3921fd 77af4ed951a7 6dcc75eaf823 f9b20c280980 1b6d0614f2c7 3486e5fe0d73 4b62ac958854 1f72b4630c4a 05c859b5a17c b2f316884260 93c0dec8ced7 fc25f79e6d18 c1a112035695 1d8b1029729c c33b5c3d0449 a4ffd27183aa bf064c332aa1 62a63953f966 034e2d7f187e 0562c756a2f2 40cb5e209b92 1b6d0614f2c7 ce1f034abb5d 7420f0cec354 e943e5e5b779 b208ae1e8232 f8b0c07e306c 0072efdaa34a 98d0d51b397c fbb5efbcc5b3 4b62ac958854 e170f65889d6 33de92ab3817 4129ea7e3fb2 0c4ce226d9fe d38820625542 ba02159e05b1 cde4f1b2a877 6ce6cc5a3203 62189de225a6 f95d0bea231b 6ca2dd348663 63fa15c4caa9 c9a53ea6e219 10e45001c2f2 094453b4e4ae 586242498a88 586242498a88 9690f01b4172 818a7ff3bf29 578830762b27 cc429363fb23 e27acd17313e eeb86a6a04e4 8d21095e8690 b73e657498f2 be6b97be2959 59c7f349c89a 93790ade6682 a65259ff0092 4357c81e10c1 a31962fbd5f3 0562c756a2f2 e259a56993f4 1b6d0614f2c7 b9699ce57810 eeb86a6a04e4 b61f1af56200 036087ac04f9 25c57acdf805 b136f6349cf3 586242498a88 54709b24b45f 2d5d7bc2df91 6b583fe13e39 798fe9915030 8e93a2273a93 1c303d15eb65 bce6be3921fd 798eee3a8bec 7d41ca882f26 93c988b67c47 2ef7c27a5df4 fea862065b74 0c222c6660f2 34186de5c52d 25c57acdf805 bb0e7ae8fdbf dafbb201715e fb2cd24a447a 6bf9c0cb01b4 0562c756a2f2 ba02159e05b1 6b304aabdcee 6d25574664d2 9cdf4a63deb0 b59e343416f7 26f7353edc2e 133d46f7ed38 9bc96abb24e5 eeb86a6a04e4 f95d0bea231b 74c7b4b972ff 10e45001c2f2 10e45001c2f2 6ce6cc5a3203 10e45001c2f2 b73e657498f2 f95d0bea231b d19b1c129f40 25c57acdf805 6101ed18e42f'
label = 'BILL'
doc_arr = [x for x in sample[1]]

word_arr = words.split(' ')
test_word_vecs = np.array([[word_2_vect(word) for word in word_arr][:20]])
#b_size_input = np.array([test_word_vecs[0] * 500])
b_size_input = np.array([test_word_vecs[0] for x in range(500)])
test_word_vecs.shape
# (1, 20, 100)

(1, 20, 100)

In [215]:
help(cell)


Help on MultiRNNCell in module tensorflow.python.ops.rnn_cell_impl object:

class MultiRNNCell(RNNCell)
 |  RNN cell composed sequentially of multiple simple cells.
 |  
 |  Example:
 |  
 |  ```python
 |  num_units = [128, 64]
 |  cells = [BasicLSTMCell(num_units=n) for n in num_units]
 |  stacked_rnn_cell = MultiRNNCell(cells)
 |  ```
 |  
 |  Method resolution order:
 |      MultiRNNCell
 |      RNNCell
 |      tensorflow.python.layers.base.Layer
 |      tensorflow.python.keras.engine.base_layer.Layer
 |      tensorflow.python.module.module.Module
 |      tensorflow.python.training.tracking.tracking.AutoTrackable
 |      tensorflow.python.training.tracking.base.Trackable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, cells, state_is_tuple=True)
 |      Create a RNN cell composed sequentially of a number of RNNCells. (deprecated)
 |      
 |      Warning: THIS FUNCTION IS DEPRECATED. It will be removed in a future version.
 |      Instructions for upda

In [214]:
# Can I do this smaller sized?
small_size = 1

with graph.as_default():
    with tf.name_scope("RNN_layers"):
        # Getting an initial state of all zeros
        small_initial_state = cell.zero_state(small_size, tf.float32)
        
    small_outputs, small_final_state = tf.nn.dynamic_rnn(cell, inputs_,
                                             initial_state=small_initial_state)
    small_predictions = tf.contrib.layers.fully_connected(small_outputs[:, -1], 
                                                          output_size, activation_fn=tf.sigmoid)
#     small_cost = tf.losses.mean_squared_error(labels_, small_predictions)
#     small_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(small_cost)
#     small_correct_pred = tf.equal(tf.cast(tf.round(small_predictions), tf.int32), labels_)
#     small_accuracy = tf.reduce_mean(tf.cast(small_correct_pred, tf.float32))
    
with tf.Session(graph=graph) as sess:
    saver.restore(sess, "checkpoints/final_sentiment.ckpt")
    test_state = sess.run(cell.zero_state(small_size, tf.float32))
    feed = {inputs_: np.array([test_word_vecs for x in range(500)]),
            labels_: np.array([[0] * output_size]),
            keep_prob: 1,
            initial_state: test_state}
    batch_acc, test_state, op, corr_pdctn, pdctns = sess.run([small_predictions], feed_dict=feed)
#     print(pdctns.shape)
#     print(pdctns.shape)
#     print(pdctns[0])   
   


    
#     #test_acc.append(batch_acc)
#     #print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

ValueError: Cannot feed value of shape (500, 1, 20, 100) for Tensor 'Placeholder:0', which has shape '(?, 20, 100)'

In [178]:
def max_prediction(pdctns):
    max_pred = max(pdctns[0])
    return np.array([int(n == max_pred) for n in pdctns[0]])



with tf.Session(graph=graph) as sess:
    saver.restore(sess, "checkpoints/final_sentiment.ckpt")
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    #for ii, (bat_test_x, bat_test_y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
    for ii in range(1):
        label_vec, inp_chunk = training_vects[ii]
        full_inp_chunk = np.array([inp_chunk for n in range(500)])

        feed = {inputs_: full_inp_chunk,
                labels_: np.array([[1] * output_size]),
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state, op, corr_pdctn, pdctns = sess.run([accuracy, final_state, outputs, correct_pred, predictions], feed_dict=feed)
        print(op.shape)
        machine_pred = max_prediction(pdctns)
        print((label_vec * machine_pred).sum())
   

(500, 20, 256)
0


In [175]:
a = np.array([1,2,3])
b = np.array([1,2,3])
(a * b)


array([1, 4, 9])

In [135]:
list(enumerate(pdctns[0]))

[(0, 0.20197132),
 (1, 0.0024269521),
 (2, 0.011229187),
 (3, 0.5622808),
 (4, 0.0050757527),
 (5, 0.0041457117),
 (6, 0.024143666),
 (7, 0.011681408),
 (8, 0.0022202134),
 (9, 0.013743609),
 (10, 0.026958704),
 (11, 0.043389678),
 (12, 0.006714314),
 (13, 0.0013355911)]

In [216]:
encoder.transform([label])

array([1])

In [137]:
encoder.classes_

array(['APPLICATION', 'BILL', 'BILL BINDER', 'BINDER',
       'CANCELLATION NOTICE', 'CHANGE ENDORSEMENT', 'DECLARATION',
       'DELETION OF INTEREST', 'EXPIRATION NOTICE',
       'INTENT TO CANCEL NOTICE', 'NON-RENEWAL NOTICE', 'POLICY CHANGE',
       'REINSTATEMENT NOTICE', 'RETURNED CHECK'], dtype=object)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [78]:
with tf.Session(graph=graph) as sess:
    tf.Print(correct_pred, [correct_pred])
    sess.run(correct_pred)

FailedPreconditionError: Attempting to use uninitialized value rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias
	 [[node rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias/read (defined at /var/folders/bc/6ycpk0y11zx4_kh4bljhdtf5dhvskt/T/tmpuf1_xpky.py:53) ]]

Original stack trace for 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias/read':
  File "/Users/boaz.reisman/.pyenv/versions/3.6.5/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/boaz.reisman/.pyenv/versions/3.6.5/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/boaz.reisman/.virtualenvs/datascience/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/boaz.reisman/.virtualenvs/datascience/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/boaz.reisman/.virtualenvs/datascience/lib/python3.6/site-packages/ipykernel

In [67]:
help(correct_pred)

Help on Tensor in module tensorflow.python.framework.ops object:

class Tensor(_TensorLike)
 |  Represents one of the outputs of an `Operation`.
 |  
 |  A `Tensor` is a symbolic handle to one of the outputs of an
 |  `Operation`. It does not hold the values of that operation's output,
 |  but instead provides a means of computing those values in a
 |  TensorFlow `tf.compat.v1.Session`.
 |  
 |  This class has two primary purposes:
 |  
 |  1. A `Tensor` can be passed as an input to another `Operation`.
 |     This builds a dataflow connection between operations, which
 |     enables TensorFlow to execute an entire `Graph` that represents a
 |     large, multi-step computation.
 |  
 |  2. After the graph has been launched in a session, the value of the
 |     `Tensor` can be computed by passing it to
 |     `tf.Session.run`.
 |     `t.eval()` is a shortcut for calling
 |     `tf.compat.v1.get_default_session().run(t)`.
 |  
 |  In the following example, `c`, `d`, and `e` are symbolic 

In [73]:
tf.print(correct_pred)

<tf.Operation 'PrintV2_1' type=PrintV2>

In [76]:
tf.Session.run()

TypeError: run() missing 2 required positional arguments: 'self' and 'fetches'